**Cross-Encoder Re-Ranking**

Install sentence-transformers python library  and import 'CrossEncoder' from this library.

In [1]:
# place your code here
from sentence_transformers import CrossEncoder

c:\Users\artur\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# place your code here

From typing  import List, Tuple

In [2]:
# place your code here
from typing import List, Tuple

Load the pre-trained cross-encoder model 'cross-encoder/ms-marco-MiniLM-L-6-v2' and save the model object to the 'model' variable. To load, use CrossEncoder() with two arguments: the first is the model name as string and the second argument is 'max_length=512'

In [3]:
# place your code here
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

Loading weights: 100%|██████████| 105/105 [00:00<00:00, 330.82it/s, Materializing param=classifier.weight]                                    
BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Use function:

In [4]:
def rerank_documents(
    query: str,  documents: List[str],  top_k: int = 10
) -> List[Tuple[int, str, float]]:
    """
    Re-rank documents based on relevance to query.
    Args:
        query: The search query
        documents: List of document texts from initial retrieval
        top_k: Number of top results to return
    Returns:
        List of (original_index, document, score) tuples, sorted by relevance
    """
    # Create query-document pairs for the cross-encoder
    # Each pair will be scored independently
    pairs = [[query, doc] for doc in documents]

    # Get relevance scores for all pairs
    # The model outputs a single score per pair
    scores = model.predict(pairs)

    # Combine with original indices and sort by score descending
    scored_docs = [
        (idx, doc, float(score))
        for idx, (doc, score) in enumerate(zip(documents, scores))
    ]
    scored_docs.sort(key=lambda x: x[2], reverse=True)

    return scored_docs[:top_k]

Simulate retrieval results: use the following data.

In [5]:
retrieved_docs = [
        "Kubernetes pod monitoring requires metrics collection from the kubelet.",
        "Docker containers can be monitored using cAdvisor metrics.",
        "To check pod status, use kubectl get pods command.",
        "Prometheus is commonly used for Kubernetes monitoring.",
        "Pod resource limits should be set in the deployment spec.",
        "OneUptime provides real-time Kubernetes monitoring dashboards.",
        "Container orchestration platforms need observability solutions.",
        "The kubectl logs command shows container output.",
    ]
query = "How do I monitor Kubernetes pods?"

Select a value for the k parameter and apply the "rerank_documents()" function to the data.
Save the returned object to the results variabl.

In [6]:
# place your code here.
results = rerank_documents(query, retrieved_docs, top_k=5)

Using the 'for idx, doc, score in results'    loop, print the values ​​of 'idx', 'doc' and 'score'

In [7]:
print("Re-ranked results:")
# place your code here
for idx, doc, score in results:
    print("Index: ", idx)
    print("Score: ", score)
    print("Doc: ", doc)

Re-ranked results:
Index:  0
Score:  6.195793628692627
Doc:  Kubernetes pod monitoring requires metrics collection from the kubelet.
Index:  5
Score:  2.8766818046569824
Doc:  OneUptime provides real-time Kubernetes monitoring dashboards.
Index:  3
Score:  2.8236851692199707
Doc:  Prometheus is commonly used for Kubernetes monitoring.
Index:  2
Score:  0.14170622825622559
Doc:  To check pod status, use kubectl get pods command.
Index:  7
Score:  -8.542131423950195
Doc:  The kubectl logs command shows container output.
